# 计算机网络结果第三次作业

作业要求：

> 请复现“第十章 海龟交易系统”中介绍的六个长期趋势跟踪系统。
>
> 请以一段时间内的真实行情数据，
>
> 1）给出如《海龟交易法则（第四版）》中“表 10-1 6 个系统的历史表现对比”的测试结果。
>
> 2）并给出相应的结果说明和分析。自己的收获和感想也可以写进去。如果不能按时完成，请写明大致花费时间，原因。
>
> 注：行情数据可以通过 AkSahre 接口获得。如果是针对股票数据，可以考虑设定账户规模是 100 万人民币，单个标的持仓上限是 10 万人民币。


## 实验设置

- 实验目的：复现《海龟交易法则（第四版）》中介绍的六个长期趋势跟踪系统，并给出测试结果。
- 实验数据：美股 ETF 数据（后复权）。
  - 标的：澳元、英镑、谷物、可可、加拿大元、原油、棉花、欧元、欧洲美元、饲牛、⻩金、铜、⺠用燃料油、无铅汽油、日元、咖啡、牛、猪、墨⻄哥比索、天然气、大豆、糖、瑞士法郎、银、中期国债、⻓期国债以及小⻨
  - 时间范围：2010-01-01 至 2024-05-20。
- 实验参数：初始资金 100 万人民币，单个标的持仓上限 10 万人民币。
- 评价指标：复合平均增长率、平均月化收益率、夏普比率、最大回撤、年化波动率、胜率、盈亏比。


## 原理和实现


In [1]:
from typing import Callable
import akshare as ak
import os.path
import pandas as pd
import pathlib
import datetime


def get_data(path: str, getter: Callable[[], pd.DataFrame], force: bool = False) -> pd.DataFrame:
    '''
    从akshare获取数据
    path: 数据文件的路径
    getter: 获取数据的函数
    force: 是否强制重新获取数据
    返回值: pd.DataFrame
    '''
    p = pathlib.Path("data") / path
    if os.path.exists(p) and not force:
        return pd.read_pickle(p)
    else:
        data = getter()
        data.to_pickle(p)
        return data


# 获取美股实时行情数据
stock = get_data('stock_us_spot_em.pkl', ak.stock_us_spot_em)
print(stock.columns)
stock.loc[:, ['名称', '代码']].to_csv('美股列表.csv')

Index(['序号', '名称', '最新价', '涨跌额', '涨跌幅', '开盘价', '最高价', '最低价', '昨收价', '总市值',
       '市盈率', '成交量', '成交额', '振幅', '换手率', '代码'],
      dtype='object')


In [2]:
# 根据符号获取指定标的的代码
codes = {
    # 澳元
    '澳元ETF-CurrencyShares': '107.FXA',

    # 英镑
    '英镑ETF-CurrencyShares': '107.FXB',

    # 谷物
    # 可可
    # 加拿大元

    # 原油
    '原油ETF-PowerShares': '107.DBO',
    # '二倍做空原油ETF-ProShares': '107.SCO',
    # '二倍做多原油ETF-ProShares': '107.UCO',
    # '美国原油基金(近12月合约平均)': '107.USL',
    # '美国布伦特原油基金': '107.BNO',
    # '美国原油基金': '107.USO',
    # '马拉松原油': '106.MPC',

    # 棉花
    # 欧元
    '二倍做空欧元ETF-ProShares': '107.EUO',
    '二倍做多欧元ETF-ProShares': '107.ULE',
    '欧元ETF-CurrencyShares': '107.FXE',

    # 欧洲美元

    # 饲牛

    # ⻩金

    # 铜
    # 107.COPX 2010-04-20
    # '铜矿ETF-Global X': '107.COPX',

    # ⺠用燃料油

    # 无铅汽油

    # 日元
    # '二倍做空日元ETF-ProShares': '107.YCS',
    '日元ETF-CurrencyShares': '107.FXY',
    # '二倍做多日元ETF-ProShares': '107.YCL',

    # 咖啡
    # 牛
    # 猪
    # 墨⻄哥比索

    # 天然气
    # '二倍做多天然气ETF-ProShares': '107.BOIL',
    # '美国天然气基金': '107.UNG',
    '天然气ETF-First Trust': '107.FCG',
    # '二倍做空天然气ETF-ProShares': '107.KOLD',

    # 大豆
    '大豆': '107.SOYB',

    # 糖

    # 瑞士法郎
    '瑞士法郎ETF-CurrencyShares': '107.FXF',

    # 银
    # 106.PAAS 2023-04-18
    # '泛美白银': '106.PAAS',

    # 中期国债
    # 105.SHY 2017-08-07
    # '1-3年国债ETF-iShares': '105.SHY',

    # ⻓期国债
    # '三倍做空20年+国债ETF-Direxion': '107.TMV',
    # '二倍做空20年+国债ETF-ProShares': '107.TBT',
    # '做空20年+国债ETF-ProShares': '107.TBF',
    '10-20年国债ETF-iShares': '107.TLH',
    # 105.TLT 2016-02-02
    # '20年+国债ETF-iShares': '105.TLT',
    '美国超长期国债ETF-Vanguard': '107.EDV',
    '美国25年+零息票国债ETF-PIMCO': '107.ZROZ',
    # '二倍做多20年+国债ETF-ProShares': '107.UBT',
    # '三倍做多20年+国债ETF-Direxion': '107.TMF',

    # 小⻨
}

data_set: dict[str, pd.DataFrame] = {}
for name, code in codes.items():
    # print(name, code)
    data = get_data(
        f'{code}.pkl',
        lambda: ak.stock_us_hist(
            symbol=code, period='daily', start_date='20000101', end_date='20240520', adjust='hfq'
        )
    )
    # print(data.loc[:, '日期'])
    if data.loc[0, '日期']:
        print(code, data.loc[0, '日期'])
    data_set.setdefault(code, data)
print(data.columns)

107.FXA 2006-06-26
107.FXB 2006-06-26
107.DBO 2007-01-05
107.EUO 2008-11-25
107.ULE 2008-11-25
107.FXE 2005-12-12
107.FXY 2007-02-13
107.FCG 2007-05-11
107.SOYB 2011-09-19
107.FXF 2006-06-26
107.TLH 2007-01-11
107.EDV 2008-01-29
107.ZROZ 2009-11-04
Index(['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率'], dtype='object')


In [3]:
# 初始资金
INITIAL = 1000000
# 单个标的最大持仓
LIMIT = 100000

In [4]:
# ATR通道突破系统

def atr_channel_breakout(n: int = 14) -> pd.DataFrame:
    '''
    计算ATR
    data: pd.DataFrame
    n: int
    返回值: pd.DataFrame
    '''
    cash = INITIAL
    stocks: dict[str, float] = {}
    for code, data in data_set.items():
        stocks.setdefault(code, 0)
        data.loc[:, '日期'] = pd.to_datetime(data.loc[:, '日期'])
        data.set_index('日期', inplace=True)
        data['真实波幅'] = data.apply(
            lambda x: max(
                x['最高'] - x['最低'],
                abs(x['最高'] - x['收盘']),
                abs(x['最低'] - x['收盘'])
            ),
            axis=1
        )
        # print(n)
        data['平均真实波幅'] = data['真实波幅'].rolling(n).mean()
        data['350日移动平均收盘价'] = data['收盘'].rolling(
            datetime.timedelta(350)).mean()
        data['通道顶部'] = data['350日移动平均收盘价'] + 7 * data['平均真实波幅']
        data['通道底部'] = data['350日移动平均收盘价'] - 3 * data['平均真实波幅']

    result = []

    days = data_set['107.FXA'][data_set['107.FXA'].index > '2020-01-01'].index

    for i in range(len(days)):
        total = cash
        for code, data in data_set.items():
            yesterday = days[i - 1]
            today = days[i]
            open_today: float = data.loc[today, '开盘']
            close_yesterday: float = data.loc[yesterday, '收盘']
            upper_bound: float = data.loc[yesterday, '通道顶部']
            lower_bound: float = data.loc[yesterday, '通道底部']
            atr = data.loc[yesterday, '平均真实波幅']

            if close_yesterday * stocks[code] > LIMIT:
                # 卖出
                to_sell = (LIMIT - stocks[code] * open_today) // open_today
                # assert to_sell >= 0, (to_sell, stocks[code], cash)
                if to_sell > 0:
                    cash += open_today * to_sell
                    stocks[code] -= to_sell
            elif close_yesterday > upper_bound:
                # 买入
                if stocks[code] * close_yesterday < LIMIT:
                    to_buy = min(
                        (LIMIT - stocks[code] *
                         close_yesterday)//close_yesterday,
                        cash // open_today
                    )
                    assert to_buy >= 0, (to_buy, stocks[code], cash)
                    stocks[code] += to_buy
                    cash -= open_today * to_buy
                    assert cash >= 0
            elif close_yesterday < lower_bound:
                # 卖出
                cash += open_today * stocks[code]
                stocks[code] = 0
            total += data.loc[today, '收盘'] * stocks[code]
        r = {
            '日期': today,
            '现金': cash,
            '资金': total,
        }
        r.update({
            code: stocks[code] for code, df in data_set.items()
        })
        result.append(r)
    return pd.DataFrame(result)

In [5]:
atr_channel_breakout()

,日期,现金,资金,107.FXA,107.FXB,107.DBO,107.EUO,107.ULE,107.FXE,107.FXY,107.FCG,107.SOYB,107.FXF,107.TLH,107.EDV,107.ZROZ
0,2020-01-02,1000000.000,1000000.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-03,1000000.000,1000000.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-06,1000000.000,1000000.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-07,800610.580,1199366.350,0.0,783.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1051.0,0.0,0.0,0.0
4,2020-01-08,800228.710,999951.770,0.0,786.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1051.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2024-05-14,467325.515,1210477.117,0.0,0.0,9699.0,4128.0,0.0,0.0,0.0,8319.0,5402.0,1036.0,0.0,0.0,0.0
1099,2024-05-15,467325.515,1210039.835,0.0,0.0,9699.0,4128.0,0.0,0.0,0.0,8319.0,5402.0,1036.0,0.0,0.0,0.0
1100,2024-05-16,467325.515,1208728.033,0.0,0.0,9699.0,4128.0,0.0,0.0,0.0,8319.0,5402.0,1036.0,0.0,0.0,0.0
1101,2024-05-17,467325.515,1212241.083,0.0,0.0,9699.0,4128.0,0.0,0.0,0.0,8319.0,5402.0,1036.0,0.0,0.0,0.0


## 实验结果与分析

### 系统表现对比

| System | CAGR% | MAR | Sharpe | Trades | W%  | Max DD | DD Length |
| :----: | :---: | :-: | :----: | :----: | :-: | :----: | :-------: |
